In [59]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install imblearn
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install pyod
!{sys.executable} -m pip install lightgbm
!{sys.executable} -m pip install catboost  
!{sys.executable} -m pip install -c alubbock graphviz pygraphviz
!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install plotly

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'alubbock'


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,precision_recall_curve,roc_curve,plot_precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import time
import matplotlib.patches as mpatches
from sklearn.metrics import confusion_matrix
import gc
import networkx as nx

gc.enable()

pd.set_option('display.max_rows',100)
pd.set_option('max_columns',None)
pd.set_option('max_colwidth',None)

In [28]:
%%time 
# pathcontext_node_types=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/code2vec/java/node_types.csv' )
# pathcontext_contexts=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/code2vec/java/path_contexts.csv' )
# pathcontext_paths=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/code2vec/java/paths.csv' )
# pathcontext_token=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/code2vec/java/tokens.csv' )

print('Train data set is loaded !')

Train data set is loaded !
Wall time: 0 ns


In [66]:
%%time 
pathcontext_node_types=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/pathcontexts/java/node_types.csv',index_col='id' )
pathcontext_contexts=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/pathcontexts/java/path_contexts.csv', header= None, delimiter= ' ' )
pathcontext_paths=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/pathcontexts/java/paths.csv', index_col='id' )
pathcontext_token=pd.read_csv('C:/Users/monju/Documents/sqa-research/features/pathcontexts/java/tokens.csv' )

print('Train data set is loaded !')

Train data set is loaded !
Wall time: 114 ms


In [54]:
filtered_node_type = pathcontext_node_types.loc[pathcontext_node_types.index =='121']
print(filtered_node_type)

Empty DataFrame
Columns: [node_type]
Index: []


C:\tools\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:122: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = op(self.values, np.asarray(other))


In [64]:
pathcontext_node_types.query('id==121').values[0][0]

'EnumDeclaration DOWN'

In [88]:
path_value = pathcontext_paths.query('id==4060').values[0][0]
print(path_value)
path_nodes = path_value.split(' ')
print(path_nodes)

for i in path_nodes:
    print(pathcontext_node_types.query('id=='+ str(i)).values[0][0])

1 37 70 60 80 81 82 18 130 3 4 5
['1', '37', '70', '60', '80', '81', '82', '18', '130', '3', '4', '5']
SimpleName UP
MethodInvocation UP
Assignment UP
ExpressionStatement UP
Block UP
TryStatement TOP
CatchClause DOWN
Block DOWN
ThrowStatement DOWN
ClassInstanceCreation DOWN
SimpleType DOWN
SimpleName DOWN


In [23]:
print('heads')
print('pathcontext node types')
print(pathcontext_node_types.head())

print('==================')

print('pathcontext paths')
print(pathcontext_paths)

print('==================')

print('pathcontext token')
print(pathcontext_token.head())

print('==================')


heads
pathcontext node types
    id                            node_type
0  121                 EnumDeclaration DOWN
1  156                       CatchClause UP
2  110  AnnotationTypeMemberDeclaration TOP
3  131            ConstructorInvocation TOP
4   43                     TypeLiteral DOWN
pathcontext paths
        id                              path
0     4060  1 37 70 60 80 81 82 18 130 3 4 5
1     2540                1 11 42 59 37 25 5
2      942                    6 71 73 17 4 8
3     1152     1 37 41 38 39 19 20 26 43 4 5
4      781                      6 111 112 28
...    ...                               ...
4123  2589           1 11 38 39 40 9 26 26 5
4124   837              1 23 74 71 73 17 4 5
4125  3394             1 15 18 30 26 26 47 5
4126   148              1 11 15 18 30 34 4 5
4127  2342                           1 63 61

[4128 rows x 2 columns]
pathcontext token
      id                                            token
0     37                                        

In [33]:
print('describe')
print('pathcontext node types')
print(pathcontext_node_types.describe())

print('==================')

print('pathcontext paths')
print(pathcontext_paths.describe())

print('==================')

print('pathcontext token')
print(pathcontext_token.describe())

describe
pathcontext node types
          node_type
count           156
unique          156
top     Modifier UP
freq              1
pathcontext paths
                     path
count                4126
unique               4126
top     1 15 18 40 29 4 5
freq                    1
pathcontext token
                                                    token
count                                               11951
unique                                              11951
top     version|.|.|final|compiler|javac|environment|j...
freq                                                    1


In [34]:
pathcontext_node_types

,node_type
id,
77,TypeDeclaration DOWN
154,CharacterLiteral DOWN
67,SuperMethodInvocation UP
73,MethodDeclaration DOWN
25,MethodInvocation TOP
81,TryStatement TOP
129,NormalAnnotation DOWN
35,ArrayType DOWN
78,MarkerAnnotation DOWN


In [35]:
pathcontext_paths

,path
id,
4058,1 37 70 60 80 81 82 18 130 3 4 5
2535,1 11 42 59 37 25 5
942,6 71 73 17 4 8
1152,1 37 41 38 39 19 20 26 43 4 5
781,6 111 112 28
442,6 71 73 17 4 5
1937,1 37 25 43 4 5
2362,1 23 15 18 40 9 5
2549,1 70 60 39 40 9 5


In [36]:
pathcontext_token

,token
id,
37,NaN
1244,payment|details
11092,upload
2972,p|to|merit|serial|number
4476,orgumsdomainmodelimmutableapplicationsapp|rules
11441,split|payment|dtos
2182,cci
8866,p|agenda|resolution|manager
10096,get|no|of|students|with|equal|or|greater|c|gpa


In [47]:
merged.columns

Index(['dot_file', 'source_file', 'label', 'node_id', 'token', 'type',
       'node_type'],
      dtype='object')

In [45]:
%%time
merged =  pd.merge(parse_descriptions, pathcontext_node_types, left_on='node_id', right_on='id')

Wall time: 161 ms


In [48]:
merged.loc[:,['label','type','node_type']]

,label,type,node_type
0,C_\Users\monju\Documents\ums_src\UMS\auth_support\src\main\java\org\ums\authorization\PermissionResolver_java,PackageDeclaration,SimpleName UP
1,C_\Users\monju\Documents\ums_src\UMS\auth_support\src\main\java\org\ums\credentials\PlainPasswordMatcher_java,PackageDeclaration,SimpleName UP
2,C_\Users\monju\Documents\ums_src\UMS\auth_support\src\main\java\org\ums\credentials\RetryLimitHashedCredentialsMatcher_java,PackageDeclaration,SimpleName UP
3,C_\Users\monju\Documents\ums_src\UMS\auth_support\src\main\java\org\ums\filter\AbstractPathMatchingFilter_java,PackageDeclaration,SimpleName UP
4,C_\Users\monju\Documents\ums_src\UMS\auth_support\src\main\java\org\ums\filter\BankCaptchaFilter_java,PackageDeclaration,SimpleName UP
...,...,...,...
317185,C_\Users\monju\Documents\ums_src\UMS\ums_webservice_library\src\main\java\org\ums\resource\MutableRecordResource_java,SimpleName,Javadoc TOP
317186,C_\Users\monju\Documents\ums_src\UMS\ums_webservice_library\src\main\java\org\ums\resource\MutableSupplierResource_java,SimpleType,Javadoc TOP
317187,C_\Users\monju\Documents\ums_src\UMS\ums_webservice_library\src\main\java\org\ums\resource\PublisherResource_java,SimpleName,Javadoc TOP
317188,C_\Users\monju\Documents\ums_src\UMS\ums_webservice_library\src\main\java\org\ums\resource\RecordResource_java,SingleMemberAnnotation,Javadoc TOP


In [30]:
print(merged)

            dot_file                                        source_file  \
0          ast_0.dot  C_\Users\monju\Documents\ums_src\UMS\auth-supp...   
1          ast_1.dot  C_\Users\monju\Documents\ums_src\UMS\auth-supp...   
2          ast_2.dot  C_\Users\monju\Documents\ums_src\UMS\auth-supp...   
3          ast_3.dot  C_\Users\monju\Documents\ums_src\UMS\auth-supp...   
4          ast_4.dot  C_\Users\monju\Documents\ums_src\UMS\auth-supp...   
...              ...                                                ...   
317185  ast_2838.dot  C_\Users\monju\Documents\ums_src\UMS\ums-webse...   
317186  ast_2839.dot  C_\Users\monju\Documents\ums_src\UMS\ums-webse...   
317187  ast_2840.dot  C_\Users\monju\Documents\ums_src\UMS\ums-webse...   
317188  ast_2842.dot  C_\Users\monju\Documents\ums_src\UMS\ums-webse...   
317189  ast_2843.dot  C_\Users\monju\Documents\ums_src\UMS\ums-webse...   

                                                    label  node_id  \
0       C_\Users\monju\Docume